# `polars_bloomberg` Examples
(c) 2024 Marek Ozana

In [1]:
# Imports
from polars_bloomberg import BQuery
from datetime import date

## Bloomberg Data Point
```python
bdp(securities: List[str],
    fields: List[str],
    overrides: Optional[Sequence] = None,
    options: Optional[Dict] = None
)
```

In [2]:
# Bloomberg Data Point
with BQuery() as bq:
    df = bq.bdp(["SPX Index", "VIX Index"], ["NAME", "PX_LAST"])
df

security,NAME,PX_LAST
str,str,f64
"""SPX Index""","""S&P 500 INDEX""",6080.5
"""VIX Index""","""Cboe Volatility Index""",13.44


## Bloomberg Data History
```python
bdh(securities: List[str],
    fields: List[str],
    start_date: date,
    end_date: date,
    overrides: Optional[Sequence] = None,
    options: Optional[Dict] = None
)

```

In [3]:
# Bloomberg Data History
with BQuery() as bq:
    df = bq.bdh(
        ["SPY US Equity", "TLT US Equity"],
        ["PX_LAST", "VOLUME"],
        start_date=date(2019, 1, 1),
        end_date=date(2019, 1, 10),
        options={"adjustmentSplit": True},
    )
df.tail(5)

security,date,PX_LAST,VOLUME
str,date,f64,f64
"""TLT US Equity""",2019-01-04,122.11,1.2970226e7
"""TLT US Equity""",2019-01-07,121.75,8.498104e6
"""TLT US Equity""",2019-01-08,121.43,7.737103e6
"""TLT US Equity""",2019-01-09,121.24,9.349245e6
"""TLT US Equity""",2019-01-10,120.46,8.22286e6


## Bloomberg Query Language
```python
bql(expression: str)
```

In [4]:
# Example: average PE ratio per sector
query = """
let(#avg_pe=avg(group(pe_ratio(), gics_sector_name()));)
get(#avg_pe) 
for(members('SPX Index'))
"""
with BQuery() as bq:
    df = bq.bql(query)
df.head(5)

ID,#avg_pe,#avg_pe.REVISION_DATE,#avg_pe.AS_OF_DATE,#avg_pe.PERIOD_END_DATE,#avg_pe.ORIG_IDS,#avg_pe.GICS_SECTOR_NAME()
str,f64,date,date,date,str,str
"""Communication Services""",26.664449,2024-11-14,2024-12-04,2024-09-30,null,"""Communication Services"""
"""Consumer Discretionary""",25.804753,2024-11-27,2024-12-04,2024-11-02,null,"""Consumer Discretionary"""
"""Consumer Staples""",19.550622,2024-12-04,2024-12-04,2024-11-02,null,"""Consumer Staples"""
"""Energy""",19.174108,2024-11-12,2024-12-04,2024-09-30,null,"""Energy"""
"""Financials""",38.044045,2024-11-26,2024-12-04,2024-09-30,null,"""Financials"""


In [11]:
# Example: Duration and ZSpread for search results
query="""
let(#dur=duration(duration_type=MODIFIED); 
    #zsprd=spread(spread_type=Z);) 
get(name(), #dur, #zsprd) 
for(filter(screenresults(type=SRCH, screen_name='@COCO'), 
           ticker in ['SEB', 'SHBASS']))
"""

with BQuery() as bq:
    df = bq.bql(query)
df

ID,name(),#dur,#dur.DATE,#zsprd,#zsprd.DATE
str,str,f64,date,f64,date
"""BW924993 Corp""","""SEB 6 ⅞ PERP""",2.259635,2024-12-04,236.72222,2024-12-04
"""YU819930 Corp""","""SEB 6 ¾ PERP""",5.400873,2024-12-04,330.171723,2024-12-04
"""YV402592 Corp""","""SEB Float PERP""",0.241753,2024-12-04,249.336,2024-12-04
"""ZQ349286 Corp""","""SEB 5 ⅛ PERP""",0.424485,2024-12-04,188.177979,2024-12-04
"""ZO703315 Corp""","""SHBASS 4 ⅜ PERP""",1.982892,2024-12-04,249.232052,2024-12-04
"""ZO703956 Corp""","""SHBASS 4 ¾ PERP""",4.972547,2024-12-04,280.078174,2024-12-04


In [17]:
# Example: Average spread for an issuer
query = """
let( 
    #bins = bins(maturity_years,[3,9,18,30],['(1) 0-3','(2) 3-9','(3) 9-18','(4) 18-30','(5) 30+']);
    #average_spread = avg(group(spread(st=oas),#bins));
)
get(#average_spread)
for(filter(bonds('NVDA US Equity', issuedby = 'ENTITY'),maturity_years != NA))
"""

with BQuery() as bq:
    df = bq.bql(query)
df.plot.bar(x="ID", y="#average_spread").show()
df.head(5)

alt.Chart(...)

ID,#average_spread,#average_spread.DATE,#average_spread.ORIG_IDS,#average_spread.#BINS
str,f64,date,str,str
"""(1) 0-3""",26.841689,2024-12-04,"""QZ552396 Corp""","""(1) 0-3"""
"""(2) 3-9""",60.393963,2024-12-04,null,"""(2) 3-9"""
"""(3) 9-18""",105.69211,2024-12-04,"""BH393780 Corp""","""(3) 9-18"""
"""(4) 18-30""",128.010624,2024-12-04,"""BH393781 Corp""","""(4) 18-30"""
"""(5) 30+""",147.747398,2024-12-04,"""BH393782 Corp""","""(5) 30+"""
